## Imports

In [1]:
import cupy as cp
import numpy as np
import gc

from tifffile import tifffile 
from pathlib import Path
from sparse_recon.sparse_deconv import sparse_deconv
from cucim.skimage.exposure import match_histograms
from cucim.skimage.transform import rescale
import napari

## Load data and normalize histograms through z-stack

Channel 1

In [26]:
#img_path = Path(r"\\169.254.203.30\sim2\2021_12_24\26_1um_beads_1e2_dilution_npattern=11_odt\2022_01_05_13;24;07_odt_reconstruction\refractive_index.tif")
#img = tifffile.imread(img_path)
#img = img[:, :, 0]
#print(img.shape)

#img_time_path = Path(r"\\169.254.203.30\sim2\2021_12_24\21_1um_beads_1e3_dilution_npattern=11_odt\2022_01_04_14;25;15_odt_reconstruction\refractive_index.tif")
img_time_path = Path(r"\\169.254.203.30\sim2\2021_12_24\26_1um_beads_1e2_dilution_npattern=11_odt\2022_01_07_17;56;30_odt_reconstruction\refractive_index.tif")
img_time = tifffile.imread(img_time_path)
img_time = img_time[:, :, 0, 20:-40, 55:-50] # trim low signal part at the edge and keep only real part
print(img_time.shape)
print(img_time[0].mean())

dxy = 0.372
dz = 1.753

(200, 17, 119, 176)
1.4070022


## Perform sparse deconvolution

In [27]:
img_time_decon = np.zeros(img_time.shape)

for t_idx in range(0,img_time.shape[0]):
    print("time %d" % t_idx)
    img = img_time[t_idx,:]
    img_temp = img - 1.407
    img_temp[img_temp<0]=0
    img_temp = img_temp / img_temp.max() 

    img_decon = cp.asnumpy(sparse_deconv(img_temp, [3, 3], prior=-1, sparse_iter=100, tcontinuity=1, fidelity=20, sparsity=3, deconv_type=0, up_sample=2))

    img_time_decon[t_idx,:] = cp.asnumpy(rescale(cp.asarray(img_decon),scale=0.5,preserve_range=True))

    del img_decon
    gc.collect()
    cp.clear_memo()

time 0
Start the Sparse deconvolution...
99 iterations done
sparse hessian time
92.921875


## View widefield, 2D-SIM w/ optical sectioning, and Sparse deconvolution of 2D-SIM in Napari

In [28]:
viewer = napari.Viewer()
viewer.add_image(img_time,scale=[dz, dxy, dxy],colormap='bop purple',name="odt",blending='additive')
#viewer.add_image(img_temp,scale=[dz, dxy, dxy],colormap='bop blue',name="odt backgrd sub",blending='additive')
viewer.add_image(img_time_decon,scale=[dz, dxy, dxy],colormap='bop orange',name="odt sparse decon",blending='additive')

viewer.scale_bar.visible=True
viewer.scale_bar.unit='um'

In [20]:
#img_time_path = Path(r"\\169.254.203.30\sim2\2021_12_24\21_1um_beads_1e3_dilution_npattern=11_odt\2022_01_04_14;25;15_odt_reconstruction\refractive_index_sparse.tif")
save_path = 0

tifffile.imwrite(str(img_time_path),img_time_decon)

## Create orthogonal views and display

In [112]:
odt_sparse_xy = np.max(img_time_decon,1)
odt_sparse_xy = np.pad(odt_sparse_xy,((0,0),(0,5),(0,5)),constant_values=1)   
odt_sparse_xz = np.max(np.transpose(img_time_decon,(0,2,1,3)),1)
odt_sparse_xz = np.pad(odt_sparse_xz,((0,0),(0,5),(0,5)),constant_values=0)   
odt_sparse_yz = np.max(np.transpose(img_time_decon,(0,3,2,1)),1)
odt_sparse_yz = np.pad(odt_sparse_yz,((0,0),(0,5),(0,5)),constant_values=0)   

odt_projection = np.concatenate((odt_sparse_xy,odt_sparse_yz),2)
odt_sparse_xz_filled = np.ones((1000,22,310))
odt_sparse_xz_filled[:,0:22,0:288] = odt_sparse_xz
odt_projection = np.concatenate((odt_projection,odt_sparse_xz_filled),1)

In [113]:
viewer = napari.Viewer()
viewer.add_image(odt_projection,scale=[dxy, dxy],colormap='gray_r',name="odt sparse decon",blending='additive')
viewer.scale_bar.visible=True
viewer.scale_bar.unit='um'